In [1]:
#https://ieeexplore.ieee.org/abstract/document/9033031
#https://www.researchgate.net/profile/Abdul-Muntakim-Rafi/publication/337366713_Image-based_Bengali_Sign_Language_Alphabet_Recognition_for_Deaf_and_Dumb_Community/links/5f047f5e92851c52d61df29e/Image-based-Bengali-Sign-Language-Alphabet-Recognition-for-Deaf-and-Dumb-Community.pdf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from tensorflow.keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
fid = drive.ListFile({'q':"title='sign.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('sign.zip')
f.keys()
!unzip sign.zip

Streaming output truncated to the last 5000 lines.
  inflating: RESIZED_DATASET/31/IMG_20180716_100811.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180716_101201.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180716_101616.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180716_102131.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180716_102548.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180716_103109.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180718_084537.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180718_085333.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180718_090000.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180718_131034.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180718_131917.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180718_163821.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180718_164400.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180718_164927.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180719_131510.jpg  
  inflating: RESIZED_DATASET/31/IMG_20180719_132110.jpg  
  inflating: RESIZED_

In [2]:
gpus = tf.config.list_logical_devices('GPU')
stg=tf.distribute.MirroredStrategy(gpus)

In [3]:
import cv2
import os
from tqdm import tqdm

folder_dir = '/content/RESIZED_DATASET'
SIZE = 224
DOWNSAMPLE_RATIO = 4
JPEG_QUALITY = 100

total_files = sum(len(files) for _, _, files in os.walk(folder_dir))

with tqdm(total=total_files, desc="Processing Images") as pbar:
    for folder in os.listdir(folder_dir):
        for file in os.listdir(os.path.join(folder_dir, folder)):
                image_path = os.path.join(folder_dir, folder, file)
                img = cv2.imread(image_path)
                img_resized = cv2.resize(img, (SIZE,SIZE))
                cv2.imwrite(image_path, img_resized)
                pbar.update(1)

Processing Images: 100%|██████████| 11061/11061 [00:35<00:00, 307.50it/s]


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [16]:
batch_size = 64
picture_size = (224, 224)
train_set = tf.keras.utils.image_dataset_from_directory(
    directory=folder_dir,
    shuffle=True,
    image_size=picture_size,
    batch_size=batch_size,
    label_mode='categorical',
    validation_split=0.2,
    subset='training',
    seed = 22
)

validation_set = tf.keras.utils.image_dataset_from_directory(
    directory=folder_dir,
    shuffle=True,
    image_size=picture_size,
    batch_size=batch_size,
    label_mode='categorical',
    validation_split=0.2,
    subset='validation',
    seed = 22
)


Found 11061 files belonging to 38 classes.
Using 8849 files for training.
Found 11061 files belonging to 38 classes.
Using 2212 files for validation.


In [17]:
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation, RandomTranslation, RandomFlip, RandomZoom
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import shutil
import random
import matplotlib.pyplot as plt
from tensorflow.keras.applications import Xception

no_of_classes = 38

with stg.scope():
    img_augmentation = Sequential(
    [
        RandomRotation(factor=0.2),
        RandomTranslation(height_factor=0.1, width_factor=0.1),
        RandomZoom(height_factor=0.2)
    ],
    name="img_augmentation"
    )

    base_model = Xception(weights="imagenet", input_shape=(SIZE, SIZE, 3), include_top=False)
    inputs = Input(shape=(SIZE, SIZE, 3))
    x = img_augmentation(inputs)
    outputs = base_model(x)

    x = GlobalAveragePooling2D()(outputs)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    predictions = Dense(no_of_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=predictions)

    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=0.001),
        metrics=['accuracy'])

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 img_augmentation (Sequentia  (None, 224, 224, 3)      0         
 l)                                                              
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_2 (Dropout)         (None, 512)               0   

In [18]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss',
                                 factor=0.1,
                                 patience=2,
                                 verbose=1)
early_stopping = EarlyStopping(monitor='val_loss',
                                patience=10,
                                verbose=1)

callbacks = [lr_scheduler, early_stopping]


In [19]:
import tensorflow as tf

model.trainable = True

history = model.fit(train_set, epochs=100, validation_data=validation_set, callbacks=callbacks,
                    steps_per_epoch=len(train_set), validation_steps=len(validation_set))


Epoch 1/100
139/139 [==============================] - 192s 1s/step - loss: 2.3320 - accuracy: 0.3473 - val_loss: 2.8116 - val_accuracy: 0.4806 - lr: 0.0010
Epoch 2/100
139/139 [==============================] - 153s 1s/step - loss: 0.8783 - accuracy: 0.7330 - val_loss: 0.9632 - val_accuracy: 0.7229 - lr: 0.0010
Epoch 3/100
139/139 [==============================] - 153s 1s/step - loss: 0.5373 - accuracy: 0.8360 - val_loss: 0.7921 - val_accuracy: 0.7798 - lr: 0.0010
Epoch 4/100
139/139 [==============================] - 153s 1s/step - loss: 0.4443 - accuracy: 0.8697 - val_loss: 0.7665 - val_accuracy: 0.8128 - lr: 0.0010
Epoch 5/100
139/139 [==============================] - 152s 1s/step - loss: 0.3876 - accuracy: 0.8867 - val_loss: 0.6279 - val_accuracy: 0.8400 - lr: 0.0010
Epoch 6/100
139/139 [==============================] - 153s 1s/step - loss: 0.3233 - accuracy: 0.9067 - val_loss: 0.3259 - val_accuracy: 0.9100 - lr: 0.0010
Epoch 7/100
139/139 [==============================] - 153

In [20]:
test_data_dir = '/content/RESIZED_TESTING_DATA'
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=picture_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)
test_loss, test_acc = model.evaluate_generator(test_generator)
print('Test accuracy:', test_acc)


Found 1520 images belonging to 38 classes.


<ipython-input-20-e884a9e4762c>:10: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model.evaluate_generator(test_generator)


Test accuracy: 0.9624999761581421
